## 获取坐标所在的城市

拖山的经度和纬度是(120.14,31.406)，兰山嘴的经度和纬度是(119.89,31.212)

In [2]:
# Python 3.9
import requests
from urllib.parse import urlencode
import csv
from datetime import datetime, timedelta

def get_area_code(app_key, lng, lat):
    """
    通过经纬度获取地区码
    :param app_key: 应用的API密钥
    :param lng: 经度
    :param lat: 纬度
    :return: 地区码(c0字段)
    """
    url = 'https://route.showapi.com/9-5'
    
    params = {
        'appKey': app_key,
        'needHourData': 0,
        'need3HourForcast': 0,
        'from': 1,
        'needAlarm': 0,
        'lng': lng,
        'lat': lat,
        'needIndex': 0,
        'needMoreDay': 0
    }
    
    encoded_params = urlencode(params)
    full_url = f'{url}?{encoded_params}'
    
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    response = requests.post(full_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        # 检查返回数据格式
        if 'showapi_res_body' in data and 'cityInfo' in data['showapi_res_body'] and 'c0' in data['showapi_res_body']['cityInfo']:
            return data['showapi_res_body']['cityInfo']['c0']
        else:
            print(f"返回数据格式错误: {data}")
            return None
    else:
        print(f"获取地区码失败: {response.status_code}")
        return None

def get_weather_history(app_key, area_code, area, start_date, end_date):
    """
    获取历史天气数据
    """
    url = f'https://route.showapi.com/9-7'
    params = {
        'appKey': app_key,
        'areaCode': area_code,
        'area': area,
        'startDate': start_date,
        'endDate': end_date
    }
    
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            # 检查返回数据格式
            if 'showapi_res_body' in data and 'list' in data['showapi_res_body']:
                return data
            else:
                print(f"返回数据格式错误: {data}")
                return None
        else:
            print(f"请求失败，状态码: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"请求异常: {e}")
        return None

def write_to_csv(data, filename):
    fieldnames = ['time', 'area', 'weather', 'max_temperature', 'min_temperature', 'aqi', 'aqiLevel', 'wind_direction', 'wind_power', 'aqiInfo']
    try:
        with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if csvfile.tell() == 0:  # 如果文件为空,写入表头
                writer.writeheader()
                
            for item in data['showapi_res_body']['list']:
                writer.writerow({
                    'time': item['time'],
                    'area': item['area'],
                    'weather': item['weather'],
                    'max_temperature': item['max_temperature'],
                    'min_temperature': item['min_temperature'],
                    'aqi': item['aqi'],
                    'aqiLevel': item['aqiLevel'],
                    'wind_direction': item['wind_direction'],
                    'wind_power': item['wind_power'],
                    'aqiInfo': item['aqiInfo']
                })
        print(f"数据已写入{filename}")
    except Exception as e:
        print(f"写入CSV文件时发生错误: {e}")

if __name__ == '__main__':
    app_key = 'e2068f39a01E42F9Afb298b3C68a0486'  # 替换为你的appKey
    
    # 定义两个地点的信息
    locations = [
        {'name': '拖山', 'lng': 120.14, 'lat': 31.406},
        {'name': '兰山嘴', 'lng': 119.89, 'lat': 31.212}
    ]
    
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2024, 7, 31)
    
    # 获取每个地点的数据
    for location in locations:
        print(f"正在获取{location['name']}的数据...")
        
        # 获取地区码
        area_code = get_area_code(app_key, location['lng'], location['lat'])
        if not area_code:
            continue
            
        filename = f'weather_data_{location["name"]}.csv'
        
        # 按月获取历史数据
        current_date = start_date
        while current_date <= end_date:
            next_month = current_date + timedelta(days=32)
            next_month = next_month.replace(day=1)
            
            print(f"获取{current_date.strftime('%Y-%m')}的数据...")
            response_json = get_weather_history(
                app_key, 
                area_code, 
                location['name'],
                current_date.strftime('%Y%m%d'),
                (next_month - timedelta(days=1)).strftime('%Y%m%d')
            )
            
            if response_json:
                write_to_csv(response_json, filename)
            
            current_date = next_month


正在获取拖山的数据...
获取2021-01的数据...
数据已写入weather_data_拖山.csv
获取2021-02的数据...
数据已写入weather_data_拖山.csv
获取2021-03的数据...
数据已写入weather_data_拖山.csv
获取2021-04的数据...
数据已写入weather_data_拖山.csv
获取2021-05的数据...
数据已写入weather_data_拖山.csv
获取2021-06的数据...
数据已写入weather_data_拖山.csv
获取2021-07的数据...
数据已写入weather_data_拖山.csv
获取2021-08的数据...
数据已写入weather_data_拖山.csv
获取2021-09的数据...
数据已写入weather_data_拖山.csv
获取2021-10的数据...
数据已写入weather_data_拖山.csv
获取2021-11的数据...
数据已写入weather_data_拖山.csv
获取2021-12的数据...
数据已写入weather_data_拖山.csv
获取2022-01的数据...
数据已写入weather_data_拖山.csv
获取2022-02的数据...
数据已写入weather_data_拖山.csv
获取2022-03的数据...
数据已写入weather_data_拖山.csv
获取2022-04的数据...
数据已写入weather_data_拖山.csv
获取2022-05的数据...
数据已写入weather_data_拖山.csv
获取2022-06的数据...
数据已写入weather_data_拖山.csv
获取2022-07的数据...
数据已写入weather_data_拖山.csv
获取2022-08的数据...
数据已写入weather_data_拖山.csv
获取2022-09的数据...
数据已写入weather_data_拖山.csv
获取2022-10的数据...
数据已写入weather_data_拖山.csv
获取2022-11的数据...
数据已写入weather_data_拖山.csv
获取2022-12的数据...
数据已写入weather_data_拖山.csv
获取2